In [1]:
from conf import download_path, db_conn, kite as kite_conf, scans_save_path
import polars as pl
from src.scans.swing_scan import add_basic_indicators
from src.scans.filter_scan import basic_filter
from datetime import datetime
from src.utils import setup_logger
import logging

setup_logger()

logger = logging.getLogger(__name__)

In [2]:
END_DATE = datetime(2025, 12, 12)

In [3]:
scan_symbol_list = (
    pl.scan_parquet(scans_save_path / "adr_stocks.parquet")
    .collect()
    .get_column("symbol")
    .to_list()
)

logger.info(f"Stocks in the Scan List {len(scan_symbol_list)}")

query = f"""
select * 
from {kite_conf["hist_table_name"]}
where symbol in {tuple(scan_symbol_list)}
"""
data = pl.read_database_uri(query=query, uri=db_conn)

basic_stock_list = basic_filter(
    data=data, symbol_list=scan_symbol_list, scan_date=END_DATE
)
query = f"""
select * 
from {kite_conf["hist_table_name"]}
where symbol in {tuple(basic_stock_list)}
"""
data = pl.read_database_uri(query=query, uri=db_conn)

2025-12-15 19:17:42 | INFO | __main__ | <module> | Stocks in the Scan List 607
2025-12-15 19:17:43 | INFO | src.scans.filter_scan | basic_filter | Number of stocks in symbol list: 607
2025-12-15 19:17:43 | INFO | src.scans.filter_scan | basic_filter | Symbols after basic filter: 165


# Reddit Comment

In [4]:
# WINDOW_SIZE = 10

# (
#     data.lazy()
#     .with_columns(
#         # Shift Columns
#         [
#             pl.col(col)
#             .shift(i)
#             .over(partition_by="symbol", order_by="timestamp", descending=False)
#             .alias(f"{col}_prev_{i}")
#             for col in ["close", "timestamp"]
#             for i in [1]
#         ]
#     )
#     # Calculate Ranges
#     .with_columns(
#         (
#             (pl.max_horizontal(pl.col("close") - pl.col("close_prev_1")))
#             - pl.min_horizontal(pl.col("low") - pl.col("close_prev_1"))
#         ).alias("small_range"),
#         (
#             pl.col("close").rolling_max(window_size=WINDOW_SIZE)
#             - pl.col("low").rolling_min(window_size=WINDOW_SIZE)
#         )
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("large_range"),
#     )
#     .with_columns(
#         (
#             (
#                 pl.col("small_range").rolling_sum(window_size=WINDOW_SIZE)
#                 / pl.col("large_range")
#             ).log()
#             / pl.lit(10).log()
#         )
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("R")
#     )
#     .with_columns(
#         ((pl.col("R") > 0.6).cast(pl.Int64).rolling_sum(window_size=5) >= 5)
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("scan")
#     )
#     .filter(pl.col("scan") == True)
#     .filter(pl.col("timestamp") == datetime(2025, 9, 4))
# ).collect()

# PullBack Exp

In [5]:
NEAR_PCT = 0.01

# 1. Define the individual comparison expressions
comparisons = [
    (pl.col(f"mid_prev_{i}")) <= pl.col(f"mid_prev_{i + 1}") for i in range(0, 10)
]

# 2. Build the Cumulative AND Logic
# We create a list of conditions that are only True if ALL previous conditions were True.
cumulative_conditions = []
current_chain = pl.lit(True)

for cond in comparisons:
    # "Current streak is alive IF it was alive before AND this condition is met"
    current_chain = current_chain & cond 
    cumulative_conditions.append(current_chain)

# 3. Sum the cumulative conditions to get the streak count
mid_down_streak_expr = pl.sum_horizontal(cumulative_conditions).alias("mid_down_streak")

# --- The rest of your pipeline remains identical ---
df = add_basic_indicators(data=data)
pullback_filter = (
    df.lazy()
    .with_columns([pl.mean_horizontal(("open", "close")).round(2).alias("mid_prev_0")])
    .with_columns(
        [
            pl.col("mid_prev_0")
            .shift(i)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias(f"mid_prev_{i}")
            for i in range(1, 11)
        ]
        + [
            (
                ((pl.col("mid_prev_0") - pl.col(col)).abs() / pl.col(col)) <= NEAR_PCT
            ).alias(f"near_{col}")
            for col in ["close_ema_9", "close_ema_21", "close_sma_50"]
        ]
    )
    .with_columns(
        # Simple count of how many times the condition was met (non-consecutive)
        pl.sum_horizontal(
            [pl.when(cond).then(1).otherwise(0) for cond in comparisons]
        ).alias("mid_down_count")
    )
    .with_columns(mid_down_streak_expr) # <--- Insert the new simplified expression here
    .filter(
        (
            (pl.col("near_close_ema_9") == True)
            | (pl.col("near_close_ema_21") == True)
            | (pl.col("near_close_sma_50") == True)
        )
        # & (pl.col("mid_down_count") > 2)
        & (pl.col("timestamp") == END_DATE)
        & (pl.col("adr_pct_20") >= 3.5)
        & (pl.col("rvol_pct") < 100)
    )
    .sort(["rvol_pct", "adr_pct_20"], descending=[False, True])
    .with_row_index(name="rank", offset=1)
).collect()

In [ ]:
pullback_filter.select(
    ['rank',
 'symbol',
 'timestamp',
#  'open',
#  'high',
 'low',
 'close',
 'volume',
 'close_sma_50',
 'close_ema_9',
 'close_ema_21',
 'volume_sma_20',
 'volume_sma_50',
 'day_range',
 'adr_pct_20',
 'rvol_pct',
 'near_close_ema_9',
 'near_close_ema_21',
 'near_close_sma_50',
 'mid_down_count',
 'mid_down_streak']).sort("symbol").to_pandas()

,rank,symbol,timestamp,low,close,volume,close_sma_50,close_ema_9,close_ema_21,volume_sma_20,volume_sma_50,day_range,adr_pct_20,rvol_pct,near_close_ema_9,near_close_ema_21,near_close_sma_50,mid_down_count,mid_down_streak
0,34,ACUTAAS,2025-12-12,1642.50,1664.00,145992,1671.93,1691.32,1710.48,310393,370915,1.0192,3.53,39.0,False,False,True,9,2
1,1,AGIIL,2025-12-12,263.10,263.95,78746,266.31,268.18,268.24,449738,838240,1.0255,4.16,9.0,True,True,True,7,4
2,5,AHLUCONT,2025-12-12,952.00,958.70,21258,957.52,976.31,974.31,359423,165803,1.0301,3.93,13.0,False,True,True,6,2
3,39,ATALREAL,2025-12-12,23.99,24.16,1742882,23.85,24.04,23.89,3340058,3732767,1.0196,4.65,47.0,True,False,False,4,1
4,44,AVANTIFEED,2025-12-12,789.20,807.10,433146,740.96,808.28,797.02,1517750,844625,1.0321,4.08,51.0,True,False,False,6,1
5,31,BELRISE,2025-12-12,158.73,159.90,2199426,156.95,160.71,160.47,6485848,5941657,1.0142,3.54,37.0,True,True,False,6,0
6,33,CARRARO,2025-12-12,521.10,543.10,73311,514.29,532.76,532.62,163583,190375,1.0516,4.04,39.0,True,True,False,6,0
7,8,CHENNPETRO,2025-12-12,916.45,927.40,744282,902.18,933.44,942.46,2426687,4311141,1.0191,4.61,17.0,True,False,False,6,2
8,32,DAVANGERE,2025-12-12,3.80,3.81,2943033,3.76,3.88,3.85,8298999,7675864,1.0211,4.10,38.0,False,True,False,7,1
9,37,ECOSMOBLTY,2025-12-12,225.52,229.82,187254,226.27,230.62,231.75,307530,414157,1.0257,4.87,45.0,True,False,False,6,0


# AMIBROKER

In [7]:
data.select(pl.col("symbol").n_unique())

symbol
u32
165


In [8]:
timeframe = 252
vol_tf = 50
base_lower_limit = 0.6
pivot_length = 5
pv_limit = 0.1

ami_scan_stocks = (
    (
        data.lazy()
        .with_columns(
            # 52 week high calculation
            pl.col("close")
            .rolling_max(window_size=timeframe)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("52_week_high"),
            # volume sma calculation
            pl.col("volume")
            .rolling_mean(window_size=vol_tf)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias(f"volume_sma_{vol_tf}"),
            # pivot high calculation
            pl.col("high")
            .rolling_max(window_size=pivot_length)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_high"),
            # pivot low calculation
            pl.col("low")
            .rolling_min(window_size=pivot_length)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_low"),
            # pivot start high
            pl.col("high")
            .shift(pivot_length - 1)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_start_high"),
        )
        .with_columns(
            # pivot width
            ((pl.col("pivot_high") - pl.col("pivot_low")) / pl.col("close")).alias(
                "pivot_width"
            )
        )
        .with_columns(
            # find pivot
            (
                (pl.col("pivot_width") < pv_limit)
                & (pl.col("pivot_high") == pl.col("pivot_start_high"))
            ).alias("is_pivot"),
            # volume dry up
            pl.all_horizontal(
                [
                    pl.col("volume").shift(i) < pl.col(f"volume_sma_{vol_tf}").shift(i)
                    for i in range(pivot_length)
                ]
            )
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("vol_dry_up"),
            # near 52 week high
            (
                (pl.col("close") < pl.col("52_week_high"))
                & (pl.col("close") > base_lower_limit * pl.col("52_week_high"))
            ).alias("near_high"),
        )
        .filter(
            pl.col("near_high") & pl.col("is_pivot") & pl.col("vol_dry_up")
            # & pl.col("vol_decreasing")
        )
    )
    .collect()
    .filter(pl.col("timestamp") == END_DATE)
    .sort("symbol")
    # .get_column("symbol")
    # .to_list()
)

# ami_final_stocks = basic_filter(data=data, symbol_list=ami_scan_stocks, scan_date=END_DATE)
# ami_final_stocks

In [9]:
ami_scan_stocks.to_pandas()

,symbol,timestamp,open,high,low,close,volume,52_week_high,volume_sma_50,pivot_high,pivot_low,pivot_start_high,pivot_width,is_pivot,vol_dry_up,near_high
0,ABDL,2025-12-12,616.00,626.50,616.00,624.80,211410,683.80,616572.10,630.15,595.85,630.15,0.054898,True,True,True
1,AGIIL,2025-12-12,267.40,269.80,263.10,263.95,78746,287.90,838240.22,276.45,263.10,276.45,0.050578,True,True,True
2,APEX,2025-12-12,262.10,269.80,262.10,266.40,528454,331.75,842370.12,275.80,251.40,275.80,0.091592,True,True,True
3,ASAHIINDIA,2025-12-12,1000.90,1017.50,992.00,996.20,179756,1066.40,273937.56,1055.00,982.50,1055.00,0.072777,True,True,True
4,CHENNPETRO,2025-12-12,926.00,933.95,916.45,927.40,744282,1096.55,4311141.28,956.00,878.45,956.00,0.083621,True,True,True
5,GOKEX,2025-12-12,857.00,857.00,835.70,848.90,256877,1249.90,515278.24,885.00,808.80,885.00,0.089763,True,True,True
6,GRMOVER,2025-12-12,447.50,457.00,445.30,450.90,553361,494.35,831032.92,472.05,439.00,472.05,0.073298,True,True,True
7,JKTYRE,2025-12-12,452.10,462.90,452.05,455.00,492893,474.00,1535082.54,466.15,438.00,466.15,0.061868,True,True,True
8,JPPOWER,2025-12-12,18.34,18.62,18.20,18.25,26331136,27.23,71067691.72,19.27,17.93,19.27,0.073425,True,True,True
9,KIRLOSENG,2025-12-12,1110.00,1148.90,1109.60,1144.60,165950,1181.45,672987.62,1154.00,1078.10,1154.00,0.066311,True,True,True
